<a href="https://colab.research.google.com/github/hariomshahu/FightDetection/blob/VideoMAE_Transformer/fight_detection_using_VideoMAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/kaggle_api_credentials/kaggle.json" ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d vulamnguyen/rwf2000/

Dataset URL: https://www.kaggle.com/datasets/vulamnguyen/rwf2000/versions/
License(s): unknown
100% 11.5G/11.5G [01:05<00:00, 234MB/s]
100% 11.5G/11.5G [01:05<00:00, 189MB/s]


In [ ]:
import os

dataset_zip = "rwf2000.zip"

destination_folder = "/content/dataset"

os.makedirs(destination_folder, exist_ok=True)

!unzip -q $dataset_zip -d $destination_folder

error:  cannot create /content/dataset/RWF-2000/train/Fight/1060314╤ДтХХ╨╜╤Е╨│╨▓╤Е╨┤╨╣╤Ж╨й╨п╤Й╨Ц╨╗╤Й╨й╨▓╤Ж╨Р╨╡╤И╨╕тХС╤Е╨╛╨┤╤Ж╨ктФд╤Е╨Ъ╨л╤ДтХС╨Ы╤ДтХЧтХв╤ПтХЭ╨Ш╤Е╨Э╨о╤Й╨Ч╨б╤ЕтФВтЦС╤ПтХЭ╨Щ_17.avi
        File name too long
error:  cannot create /content/dataset/RWF-2000/train/Fight/╤ДтХХ╨┤╤Е╨╡тФВ╤Е╨╜╨а╤ЕтХС╨з╤Й╨Ч╨Ь╤Ж╨Щ╨г╤Ж╨отХв╤ПтХЭ╨Ь╤Е╨ЭтФд╤ЕтХЭ╨е╤Е╨Я╨б╤Ж╨етЦС╤Е╨а╨Э╤З╨дтХЦ╤Е╨╜╨а╤Е╨ЧтХС╤Ж╨Щ╨Ы╤ПтХЭ╨Ь╤Ж╨и╨┐╤ДтХЧ╨Р╤ДтХг╨Ш╤Е╨Ю╨п╤Е╨л╨░╤ПтХЭ╨п╤З╨л╨б╤Ж╨Ю╨╖╤Ж╨Ы╨Э╤ДтХХ╨Ы╤Е╨Х╨╕╤З╨╕╨Ы_urlgot_220.avi
        File name too long
error:  cannot create /content/dataset/RWF-2000/train/Fight/╤ДтХХ╨┤╤Е╨╡тФВ╤Е╨╜╨а╤ЕтХС╨з╤Й╨Ч╨Ь╤Ж╨Щ╨г╤Ж╨отХв╤ПтХЭ╨Ь╤Е╨ЭтФд╤ЕтХЭ╨е╤Е╨Я╨б╤Ж╨етЦС╤Е╨а╨Э╤З╨дтХЦ╤Е╨╜╨а╤Е╨ЧтХС╤Ж╨Щ╨Ы╤ПтХЭ╨Ь╤Ж╨и╨┐╤ДтХЧ╨Р╤ДтХг╨Ш╤Е╨Ю╨п╤Е╨л╨░╤ПтХЭ╨п╤З╨л╨б╤Ж╨Ю╨╖╤Ж╨Ы╨Э╤ДтХХ╨Ы╤Е╨Х╨╕╤З╨╕╨Ы_urlgot_221.avi
        File name too long
error:  cannot create /content/dataset/RWF-2000/train/Fight/╤Е╨а╨Ь╤И╨▒╨Ь╤И╨Р╨С╤Ж╨нтФР╤Е╨Я╨╗╤Ж╨н╨╡╤ЕтХХ╨╛╤ЖтФдтХЫ╤Ж╨Щ╨г╤Ж╨отХв╤ПтХЭ╨Ь╤ДтХХ╨Ы╤ДтХХ╨Р╤З╨╖╨в╤Ж╨м╨Щ╤ДтХСтХС╤И╨╖╨С╤Ж╨╜тХЧ╤З╨╡╨

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import VideoMAEImageProcessor, VideoMAEForVideoClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import OneCycleLR
from tqdm.auto import tqdm

class VideoDataset(Dataset):
    def __init__(self, video_paths, labels, image_processor, num_frames=16, is_training=True):
        self.video_paths = video_paths
        self.labels = labels
        self.image_processor = image_processor
        self.num_frames = num_frames
        self.is_training = is_training

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = self.labels[idx]

        # Load and preprocess video frames
        frames = self._load_video(video_path)

        # Ensure we have exactly num_frames
        if len(frames) < self.num_frames:
            # If we don't have enough frames, duplicate the last frame
            last_frame = frames[-1] if frames else Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))
            while len(frames) < self.num_frames:
                frames.append(last_frame)
        elif len(frames) > self.num_frames:
            # If we have too many frames, sample evenly
            indices = np.linspace(0, len(frames)-1, self.num_frames, dtype=int)
            frames = [frames[i] for i in indices]

        # Process frames
        processed = self.image_processor(frames, return_tensors="pt")

        return {
            'pixel_values': processed['pixel_values'][0],
            'label': torch.tensor(label, dtype=torch.long)
        }

    def _load_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frames = []

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                # Convert to RGB
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Resize to 224x224 (VideoMAE default size)
                frame = cv2.resize(frame, (224, 224))

                # Apply augmentations during training
                if self.is_training:
                    # Random horizontal flip
                    if np.random.random() > 0.5:
                        frame = cv2.flip(frame, 1)

                    # Random brightness
                    alpha = 0.8 + np.random.random() * 0.4  # 0.8 to 1.2
                    frame = np.clip(frame * alpha, 0, 255).astype(np.uint8)

                    # Random rotation
                    if np.random.random() > 0.5:
                        angle = np.random.uniform(-15, 15)
                        M = cv2.getRotationMatrix2D((112, 112), angle, 1.0)
                        frame = cv2.warpAffine(frame, M, (224, 224))

                # Convert to PIL Image
                frame = Image.fromarray(frame)
                frames.append(frame)

        except Exception as e:
            print(f"Error loading video {video_path}: {str(e)}")

        finally:
            cap.release()

        # If no frames were read, create a dummy frame
        if not frames:
            dummy_frame = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))
            frames = [dummy_frame] * self.num_frames

        return frames

def prepare_data(data_dir):
    train_dir = os.path.join(data_dir, 'train')
    train_fight_dir = os.path.join(train_dir, 'Fight')
    train_nonfight_dir = os.path.join(train_dir, 'NonFight')

    train_fight_videos = [os.path.join(train_fight_dir, f) for f in os.listdir(train_fight_dir) if f.endswith('.avi')]
    train_nonfight_videos = [os.path.join(train_nonfight_dir, f) for f in os.listdir(train_nonfight_dir) if f.endswith('.avi')]

    train_paths = train_fight_videos + train_nonfight_videos
    train_labels = [1] * len(train_fight_videos) + [0] * len(train_nonfight_videos)

    val_dir = os.path.join(data_dir, 'val')
    val_fight_dir = os.path.join(val_dir, 'Fight')
    val_nonfight_dir = os.path.join(val_dir, 'NonFight')

    val_fight_videos = [os.path.join(val_fight_dir, f) for f in os.listdir(val_fight_dir) if f.endswith('.avi')]
    val_nonfight_videos = [os.path.join(val_nonfight_dir, f) for f in os.listdir(val_nonfight_dir) if f.endswith('.avi')]

    val_paths = val_fight_videos + val_nonfight_videos
    val_labels = [1] * len(val_fight_videos) + [0] * len(val_nonfight_videos)

    return train_paths, train_labels, val_paths, val_labels

def train_model(train_dataset, val_dataset, num_epochs=20, batch_size=2):
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,  # Removed multiprocessing to fix warnings
        pin_memory=True if torch.cuda.is_available() else False
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        num_workers=0,  # Removed multiprocessing to fix warnings
        pin_memory=True if torch.cuda.is_available() else False
    )

    print(f"\nTraining on {len(train_dataset)} samples, validating on {len(val_dataset)} samples")

    model = VideoMAEForVideoClassification.from_pretrained(
        "MCG-NJU/videomae-base-finetuned-kinetics",
        num_labels=2,
        label2id={"NonFight": 0, "Fight": 1},
        id2label={0: "NonFight", 1: "Fight"},
        ignore_mismatched_sizes=True
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Initialize optimizer with lower learning rate and separate parameter groups
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']

    optimizer_grouped_parameters = [
        {
            'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay) and 'classifier' not in n],
            'weight_decay': 0.01,
            'lr': 1e-5
        },
        {
            'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay) and 'classifier' not in n],
            'weight_decay': 0.0,
            'lr': 1e-5
        },
        {
            'params': [p for n, p in param_optimizer if 'classifier' in n],
            'weight_decay': 0.01,
            'lr': 5e-5
        }
    ]

    optimizer = torch.optim.AdamW(optimizer_grouped_parameters)

    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer,
        T_0=len(train_loader),  # Restart every epoch
        T_mult=2,  # Double the restart interval after each restart
        eta_min=1e-6  # Minimum learning rate
    )

    best_val_accuracy = 0
    best_val_f1 = 0
    patience = 5
    patience_counter = 0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []

        progress_bar = tqdm(train_loader, desc="Training")
        for batch in progress_bar:
            pixel_values = batch['pixel_values'].to(device)
            labels = batch['label'].to(device)

            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()
            predictions = torch.argmax(outputs.logits, dim=-1)
            train_preds.extend(predictions.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

            progress_bar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'lr': f"{scheduler.get_last_lr()[0]:.2e}"
            })

        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = accuracy_score(train_labels, train_preds)

        print(f"\nAverage training loss: {avg_train_loss:.4f}")
        print(f"Training accuracy: {train_accuracy:.4f}")

        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []

        with torch.no_grad():
            for batch in val_loader:
                pixel_values = batch['pixel_values'].to(device)
                labels = batch['label'].to(device)

                outputs = model(pixel_values=pixel_values, labels=labels)
                val_loss += outputs.loss.item()

                predictions = torch.argmax(outputs.logits, dim=-1)
                val_preds.extend(predictions.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(
            val_labels, val_preds, average='binary'
        )

        print(f"\nValidation Metrics:")
        print(f"Loss: {val_loss/len(val_loader):.4f}")
        print(f"Accuracy: {val_accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_val_f1 = f1
            model.save_pretrained("best_model")
            print(f"\nNew best model saved!")
            print(f"Best validation accuracy: {val_accuracy:.4f}")
            print(f"Best validation F1-score: {f1:.4f}")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered after {epoch+1} epochs")
                break

    return model

    best_val_accuracy = 0
    best_val_f1 = 0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        model.train()
        if epoch < 5:  # Freeze backbone for first 5 epochs
            model.videomae.eval()

        train_loss = 0
        train_preds = []
        train_labels = []

        progress_bar = tqdm(train_loader, desc="Training")
        for batch in progress_bar:
            pixel_values = batch['pixel_values'].to(device)
            labels = batch['label'].to(device)

            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()
            predictions = torch.argmax(outputs.logits, dim=-1)
            train_preds.extend(predictions.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

            progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})

        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = accuracy_score(train_labels, train_preds)

        print(f"\nAverage training loss: {avg_train_loss:.4f}")
        print(f"Training accuracy: {train_accuracy:.4f}")

        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []

        progress_bar = tqdm(val_loader, desc="Validation")
        with torch.no_grad():
            for batch in progress_bar:
                pixel_values = batch['pixel_values'].to(device)
                labels = batch['label'].to(device)

                outputs = model(pixel_values=pixel_values, labels=labels)
                val_loss += outputs.loss.item()

                predictions = torch.argmax(outputs.logits, dim=-1)
                val_preds.extend(predictions.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_accuracy = accuracy_score(val_labels, val_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(val_labels, val_preds, average='binary')

        print(f"\nValidation Metrics:")
        print(f"Loss: {val_loss/len(val_loader):.4f}")
        print(f"Accuracy: {val_accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_val_f1 = f1
            model.save_pretrained("best_model")
            print(f"\nNew best model saved!")
            print(f"Best validation accuracy: {val_accuracy:.4f}")
            print(f"Best validation F1-score: {f1:.4f}")

    return model

def main():
    torch.manual_seed(42)
    np.random.seed(42)

    print("GPU Available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("GPU:", torch.cuda.get_device_name(0))
        print("Memory:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")

    image_processor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
    print("\nImage Processor Config:", image_processor.image_mean, image_processor.image_std)

    print("\nPreparing datasets...")
    data_dir = '/content/dataset/RWF-2000'
    train_paths, train_labels, val_paths, val_labels = prepare_data(data_dir)
    print(f"Found {len(train_paths)} training videos and {len(val_paths)} validation videos")

    train_dataset = VideoDataset(train_paths, train_labels, image_processor, is_training=True)
    val_dataset = VideoDataset(val_paths, val_labels, image_processor, is_training=False)
    print("Datasets created successfully")

    print("\nStarting training...")
    model = train_model(train_dataset, val_dataset)
    print("Training completed!")

if __name__ == "__main__":
    main()

GPU Available: True
GPU: NVIDIA A100-SXM4-40GB
Memory: 42.481811456 GB

Image Processor Config: [0.485, 0.456, 0.406] [0.229, 0.224, 0.225]

Preparing datasets...
Found 1576 training videos and 400 validation videos
Datasets created successfully

Starting training...

Training on 1576 samples, validating on 400 samples


Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base-finetuned-kinetics and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([400]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([400, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/20
----------


Training:   0%|          | 0/788 [00:00<?, ?it/s]


Average training loss: 0.4875
Training accuracy: 0.8236

Validation Metrics:
Loss: 0.2653
Accuracy: 0.8825
Precision: 0.8462
Recall: 0.9350
F1-score: 0.8884

New best model saved!
Best validation accuracy: 0.8825
Best validation F1-score: 0.8884

Epoch 2/20
----------


...

Training:   0%|          | 0/788 [00:00<?, ?it/s]


Average training loss: 0.3577
Training accuracy: 0.9226

Validation Metrics:
Loss: 0.1894
Accuracy: 0.9325
Precision: 0.9727
Recall: 0.8900
F1-score: 0.9295

Epoch 5/20
----------


Training:   0%|          | 0/788 [00:00<?, ?it/s]


Average training loss: 0.2344
Training accuracy: 0.9454

Validation Metrics:
Loss: 0.2009
Accuracy: 0.9425
Precision: 0.9836
Recall: 0.9000
F1-score: 0.9399

New best model saved!
Best validation accuracy: 0.9425
Best validation F1-score: 0.9399

Epoch 6/20
----------


Training:   0%|          | 0/788 [00:00<?, ?it/s]